In [ ]:
#| default_exp core

# pshnb IPython magic
> Provides `psh` persistent bash magics in Jupyter and IPython

In [ ]:
#| export
from fastcore.utils import *
import pexpect, re, os, shutil, time
from pexpect import TIMEOUT
from pathlib import Path
from getpass import getpass
from IPython.core.magic import register_cell_magic, no_var_expand
from IPython.display import display, Javascript
from IPython.paths import get_ipython_dir
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.magic_arguments import magic_arguments, argument

In [ ]:
__file__ = './00_core.ipynb'

## Foundations

In [ ]:
env = dict(os.environ, TERM='dumb', PS1='$', PS2='$')
eshell = os.environ['SHELL']
sh = pexpect.spawn(eshell, encoding='utf-8', env=env)

This cell creates the initial shell process using `pexpect.spawn()` with a basic environment configuration. The key setup includes:

- **Environment variables**: Sets `TERM='dumb'` to prevent terminal formatting issues, and standardizes prompts with `PS1='$'` and `PS2='$'`
- **Shell spawning**: Creates a persistent shell process using the user's default shell (`$SHELL`) with UTF-8 encoding
- **Process persistence**: The `sh` object maintains a continuous connection to the shell, allowing commands to be sent and responses captured across multiple interactions

PS1 is the primary shell prompt (what you see when the shell is ready for a command), and PS2 is the secondary prompt (shown when a command spans multiple lines or is incomplete). `pexpect` is a Python library for controlling interactive command-line programs by automating the sending of inputs and reading of outputs. It's particularly useful for automating tasks that normally require human interaction. `spawn` is pexpect's main function that starts a child process (like a shell) and returns a pexpect object that can communicate with it. Unlike `subprocess`, spawn maintains an interactive session - you can send commands, wait for responses, and continue the conversation with the same process instance.

This establishes the foundation for persistent shell state - variables, directory changes, and other shell modifications will persist between command executions.

In [ ]:
env = dict(os.environ, TERM='dumb', PS1='', PS2='')
sh = pexpect.spawn(eshell, encoding='utf-8', env=env)
sh.sendline('stty -echo')
sh.readline()
echo = os.urandom(8).hex()
echo_re = re.compile(fr'^{echo}\s*$', flags=re.MULTILINE)
sh.sendline(f'export PS1=""')
sh.sendline('set +o vi +o emacs')
sh.sendline('echo '+echo)
sh.expect(echo_re, timeout=2)

0

This cell implements a more sophisticated shell setup that addresses output formatting and command echo issues:

- **Echo suppression**: Uses `stty -echo` to prevent command echoing, which is crucial for clean output capture
- **Empty prompts**: Sets `PS1=""` and `PS2=""` to eliminate prompt strings that would interfere with output parsing
- **Line editing disabled**: Runs `set +o vi +o emacs` to disable interactive line editing features that can cause issues in programmatic shell interaction
- **Synchronization mechanism**: Establishes a unique echo token using `os.urandom(8).hex()` and corresponding regex pattern for reliable command completion detection

The echo token system is particularly important - it provides a reliable way to detect when a command has finished executing by sending a unique marker and waiting for it to appear in the output.

In [ ]:
sh.sendline('ls | head -3')
sh.sendline('echo '+echo)
sh.expect(echo_re, timeout=2)
print(sh.before)

00_core.ipynb
AGENTS.md
CHANGELOG.bak



This cell demonstrates the practical application of the configured shell for command execution and output capture:

- **Command execution**: Sends `ls | head -3` to list the first three files in the current directory
- **Completion detection**: Uses the echo token mechanism (`echo [random_hex]`) followed by `sh.expect(echo_re)` to reliably detect when the command has finished
- **Clean output extraction**: Captures the command output via `sh.before`, which contains everything received before the expected echo token

This pattern shows how the kernel can execute arbitrary bash commands while maintaining clean separation between command output and shell control mechanisms. The timeout parameter (2 seconds) provides protection against hanging commands, which is essential for a responsive interactive environment.

## ShellInterpreter -

In [ ]:
#| export
class ShellInterpreter:
    def __init__(self, debug=False, timeout=4, shell_path='bash', sudo=False, dumb=True, ssh=None):
        self.debug,self.timeout = debug,timeout
        if sudo: shell_path = 'sudo -i ' + shell_path
        env = dict(os.environ, TERM='dumb' if dumb else 'xterm')
        self.sh = pexpect.spawn('bash' if ssh else shell_path, encoding='utf-8', env=env)
        if ssh: self.sh.sendline(f'ssh -tt {ssh} {shell_path} --norc --noprofile')
        if isinstance(sudo,str):
            self.sh.expect('password')
            self.sh.sendline(sudo)
            self.sh.expect('\n')
        self.setup()
        
    def setup(self):
        self.sh.sendline('stty -echo')
        self.sh.expect('\n')
        self.echo = os.urandom(8).hex()
        self.echo_re = re.compile(fr'^{self.echo}\s*$', flags=re.MULTILINE)
        self.sh.sendline('PROMPT=""')
        self.sh.sendline('PROMPT2=""')
        self.sh.sendline('export PS1=""')
        self.sh.sendline('export PS2=""')
        self.sh.sendline('set +o vi +o emacs')
        self.wait_echo()

    def wait_echo(self, timeout=None):
        self.sh.sendline('echo')
        self.sh.sendline('echo '+self.echo)
        self.sh.expect(self.echo_re, timeout=timeout)
        return self.sh.before.lstrip('\r\n').rstrip()

    def _ex(self, s, timeout=None):
        if timeout is None: timeout=self.timeout
        if self.debug: print('#', s)
        self.sh.sendline(s)
        res = self.wait_echo(timeout=timeout)
        return res
        
    def __call__(self, cmd, timeout=None):
        output = self._ex(cmd.rstrip(), timeout=timeout)
        # return output.replace(cmd + '\r\n', '', 1).rstrip()
        return output.rstrip()

In [ ]:
sh = ShellInterpreter()
print(sh('ls | head -3'))

00_core.ipynb
AGENTS.md
CHANGELOG.bak


NB: requires passwordless sudo to use this:

In [ ]:
sh = ShellInterpreter(sudo=True)

sh('cd')
print(sh('pwd'))
sh('cd ..')
print(sh('pwd'))
print(sh('whoami'))

/var/root


/var


root


In [ ]:
#| export
@patch
def sync_cwd(self:ShellInterpreter):
    "Sync session's cwd to match pshnb's cwd"
    _cwd = self._ex('pwd').strip()
    if _cwd != os.getcwd(): os.chdir(_cwd)

## PshMagic -

In [ ]:
#| export
def shell_replace(s, shell=None):
    "Replace `@{var}` refs in `s` with their variable values, if they exist"
    if not shell: shell = get_ipython()
    def f(m): return str(shell.user_ns.get(m[1], m[0]))
    return re.sub(r'\@{(\w+?)}', f, s)

In [ ]:
b = 1

a = '''asdf
$@{b} @{aa}
fdsa'''

print(shell_replace(a))

asdf
$1 @{aa}
fdsa


In [ ]:
#| export
class PshMagic:
    def __init__(self, shell, sudo=False, timeout=2, expand=True, o=None, ssh=None): store_attr()
    def help (self): self.bash.parser.print_help()
    def reset(self, ssh=None): 
        self.ssh = ssh
        self.o = ShellInterpreter(sudo=self.sudo, timeout=self.timeout, ssh=ssh)

    def _xpand(self, expand=False): self.expand = expand
    def _sudo(self, sudo=False):
        self.sudo = sudo
        self.o = None
    def _timeout(self, timeout=5):
        self.timeout = timeout
        self.o = None

In [ ]:
#| export
@patch
@magic_arguments()
@argument('-h', '--help',      action='store_true', help='Show this help')
@argument('-r', '--reset',   nargs='?', const=True, help='Reset the shell interpreter (optionally run ssh)')
@argument('-o', '--obj',       action='store_true', help='Return the `ShellInterpreter`')
@argument('-x', '--expand',    action='store_true', help='Enable variable expansion')
@argument('-X', '--no-expand', action='store_true', help='Disable variable expansion')
@argument('-s', '--sudo',    nargs='?', const=True, help='Enable sudo')
@argument('-S', '--no-sudo',   action='store_true', help='Disable sudo')
@argument('-c', '--cwd',       action='store_true', help="Sync session's cwd to match pshnb's cwd")
@argument('-e', '--escape',  nargs='+', help='SSH escape code and optional command')
@argument('-t', '--timeout',  type=int, help='Set timeout in seconds')
@argument('command', nargs='*', help='The command to run')
@no_var_expand
def bash(self:PshMagic, line, cell=None):
    "Run line or cell in persistent shell"
    if not cell and not line: line = 'echo'
    if cell: cell = shell_replace(cell, self.shell)
    if line: line = shell_replace(line, self.shell)
    args = self.bash.parser.parse_args(line.split())
    if args.command and not self.o: self.reset(self.ssh)
    elif not cell:
        if args.expand:    return self._xpand(True)
        if args.no_expand: return self._xpand(False)
        if args.sudo:      return self._sudo (args.sudo if isinstance(args.sudo, str) else True)
        if args.no_sudo:   return self._sudo (False)
        if args.timeout:   return self._timeout(args.timeout)
        if args.reset:     return self.reset(args.reset if isinstance(args.reset, str) else None)
        if args.help:      return self.help()
        if args.obj:       return self.o
        if args.cwd:       return self.o.sync_cwd()
        if args.escape:
            escargs = ' '.join(args.escape[1:]) if len(args.escape)>1 else None
            return self.o.ssh_escape(args.escape[0], escargs)
        if args.command: cell = ' '.join(args.command)
    if not cell and line: cell=line
    disp = True
    if cell.endswith(';'): disp,cell = False,cell[:-1]
    try: res = self.o(cell) or None
    except Exception as e:
        self.o = None
        raise e from None
    if disp and res: print(res)

In [ ]:
#| export
def create_magic(shell=None):
    if not shell: shell = get_ipython()
    magic = PshMagic(shell)
    shell.register_magic_function(magic.bash, magic_name='bash', magic_kind='line_cell')

In [ ]:
# Only required if you don't load the extension
create_magic()

In [ ]:
%bash ls | head -3

00_core.ipynb
AGENTS.md
CHANGELOG.bak


In [ ]:
%bash ls -h

::

  %bash [-h] [-r [RESET]] [-o] [-x] [-X] [-s [SUDO]] [-S] [-c]
            [-e ESCAPE [ESCAPE ...]] [-t TIMEOUT]
            [command ...]

Run line or cell in persistent shell

positional arguments:
  command               The command to run

options:
  -h, --help            Show this help
  -r <[RESET]>, --reset <[RESET]>
                        Reset the shell interpreter (optionally run ssh)
  -o, --obj             Return the `ShellInterpreter`
  -x, --expand          Enable variable expansion
  -X, --no-expand       Disable variable expansion
  -s <[SUDO]>, --sudo <[SUDO]>
                        Enable sudo
  -S, --no-sudo         Disable sudo
  -c, --cwd             Sync session's cwd to match pshnb's cwd
  -e <ESCAPE [ESCAPE ...]>, --escape <ESCAPE [ESCAPE ...]>
                        SSH escape code and optional command
  -t TIMEOUT, --timeout TIMEOUT
                        Set timeout in seconds


In [ ]:
%bash pwd

/Users/jhoward/aai-ws/pshnb


In [ ]:
%bash cd ..

In [ ]:
%bash pwd

/Users/jhoward/aai-ws


In [ ]:
!pwd

/Users/jhoward/aai-ws/pshnb


Heredoc example:

In [ ]:
%%bash
cat > tmp.foo << EOF
hi
there
EOF

In [ ]:
%bash cat tmp.foo

hi
there


In [ ]:
%bash rm tmp.foo

In [ ]:
%bash ls | head -3

ContextKit
MonsterUI
README.md


In [ ]:
%bash -c

In [ ]:
os.getcwd()

'/Users/jhoward/aai-ws'

In [ ]:
n = 2

In [ ]:
%bash echo @{n}

2


In [ ]:
%bash ls | head -@{n}

ContextKit
MonsterUI


In [ ]:
%%bash
echo starting
(sleep 1; echo finished) &

starting
[1] 51853


In [ ]:
time.sleep(1.1)

In [ ]:
%bash

finished

[1]+  Done                       ( sleep 1; echo finished )


In [ ]:
%bash -h

::

  %bash [-h] [-r [RESET]] [-o] [-x] [-X] [-s [SUDO]] [-S] [-c]
            [-e ESCAPE [ESCAPE ...]] [-t TIMEOUT]
            [command ...]

Run line or cell in persistent shell

positional arguments:
  command               The command to run

options:
  -h, --help            Show this help
  -r <[RESET]>, --reset <[RESET]>
                        Reset the shell interpreter (optionally run ssh)
  -o, --obj             Return the `ShellInterpreter`
  -x, --expand          Enable variable expansion
  -X, --no-expand       Disable variable expansion
  -s <[SUDO]>, --sudo <[SUDO]>
                        Enable sudo
  -S, --no-sudo         Disable sudo
  -c, --cwd             Sync session's cwd to match pshnb's cwd
  -e <ESCAPE [ESCAPE ...]>, --escape <ESCAPE [ESCAPE ...]>
                        SSH escape code and optional command
  -t TIMEOUT, --timeout TIMEOUT
                        Set timeout in seconds


Reset the interpreter:

In [ ]:
%bash -r

In [ ]:
%bash pwd

/Users/jhoward/aai-ws


In [ ]:
%bash echo $SHELL

/opt/homebrew/bin/bash


sudo:

In [ ]:
%bash -s

In [ ]:
%bash whoami

root


no sudo:

In [ ]:
%bash -S

In [ ]:
%bash whoami

jhoward


timeout:

In [ ]:
%bash -t 1

In [ ]:
try: get_ipython().run_line_magic('bash', 'sleep 2')
except TIMEOUT: print("timed out")

timed out


## ssh

In [ ]:
sh = ShellInterpreter(ssh='ev2')
print(sh('ls | head -3'))

go
Maildir
quarto-1.7.32-linux-amd64.deb


In [ ]:
passw = getpass()

In [ ]:
sh = ShellInterpreter(ssh='ev2', sudo=passw)
print(sh('pwd'))
print(sh('whoami'))

/root


root


In [ ]:
%bash -r

In [ ]:
%bash hostname

jphmacbook.local


In [ ]:
%bash -r ev2

In [ ]:
%bash hostname

ev2.answer.ai


In [ ]:
%bash whoami

jhoward


In [ ]:
%bash -s @{passw}

In [ ]:
%bash -r ev2

In [ ]:
%bash hostname

ev2.answer.ai


In [ ]:
%bash whoami

root


In [ ]:
#| export
def load_ipython_extension(ipython):
    "Required function for creating magic"
    create_magic(shell=ipython)

In [ ]:
#| export
def create_ipython_config():
    "Called by `pshnb_install` to install magic"
    ipython_dir = Path(get_ipython_dir())
    cf = ipython_dir/'profile_default'/'ipython_config.py'
    cf.parent.mkdir(parents=True, exist_ok=True)
    if cf.exists() and 'pshnb' in cf.read_text(): return print('pshnb already installed!')
    with cf.open(mode='a') as f: f.write("\nc.InteractiveShellApp.extensions.append('pshnb.core')\n\n")
    print(f"Jupyter config updated at {cf}")

## Export -

In [ ]:
#| hide
#| eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()